In [1]:
# Importing required libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

from cleaning import clean_data
from evaluate import qwk


from sklearn import linear_model

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, VarianceThreshold

from sklearn import ensemble


In [2]:
# Generating 3 datasets that reduce collinearity in the features
# Load Data
X_train, y_train, X_test, y_test = clean_data('')

# PCA Decomposition
pca = PCA(svd_solver='full')
pcaX_train = pca.fit_transform(X_train)
pcaX_test = pca.transform(X_test)

# Select K Best
kb = SelectKBest()
kbX_train = kb.fit_transform(X_train, y_train)
kbX_test = kb.transform(X_test)

# Variance Threshold
vt = VarianceThreshold(threshold=(.8 * (1 - .8)))
vtX_train = vt.fit_transform(X_train, y_train)
vtX_test = vt.transform(X_test)

print(f'Training = {X_train.shape}')
print(f'Testing = {X_test.shape}')
# Create dict to store outcomes

results = {}

Training = (10495, 335)
Testing = (4498, 335)


In [6]:
# Testing which dataset produces the best result on the linear regressor
controlLogReg = ensemble.GradientBoostingRegressor()
kbLogReg = ensemble.GradientBoostingRegressor()
pcaLogReg = ensemble.GradientBoostingRegressor()
vtLogReg = ensemble.GradientBoostingRegressor()

controlLogReg.fit(X_train, y_train)
kbLogReg.fit(kbX_train, y_train)
pcaLogReg.fit(pcaX_train, y_train)
vtLogReg.fit(vtX_train, y_train)

print("============= control ======================")
print("train acc: " + str(controlLogReg.score(X_train, y_train)))
print("test acc: " + str(controlLogReg.score(X_test, y_test)))

print("============= K best ======================")
print("train acc: " + str(kbLogReg.score(kbX_train, y_train)))
print("test acc: " + str(kbLogReg.score(kbX_test, y_test)))

print("============= PCA Decomposition ======================")
print("train acc: " + str(pcaLogReg.score(pcaX_train, y_train)))
print("test acc: " + str(pcaLogReg.score(pcaX_test, y_test)))

print("============= Variance Threshold ======================")
print("train acc: " + str(vtLogReg.score(vtX_train, y_train)))
print("test acc: " + str(vtLogReg.score(vtX_test, y_test)))

============= control ======================
train acc: 0.2004869603016015
test acc: 0.15395712010749807
============= K best ======================
train acc: 0.170004889558243
test acc: 0.14088230357018072
============= PCA Decomposition ======================
train acc: 0.2501164710083923
test acc: 0.11180560345203883
============= Variance Threshold ======================
train acc: 0.1894034186214375
test acc: 0.1488303067115362
